In [1]:
# run the following command in terminal
# gcloud auth login
# gcloud auth application-default login


In [1]:
from google.cloud import bigquery
import pandas as pd

# Replace 'your-project-id' with your actual Google Cloud project ID
project_id = '755693413331'

# Initialize the BigQuery client with the project ID
client = bigquery.Client(project=project_id)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Pull query

In [2]:
import dask.dataframe as dd

# Define your query
query = """
    SELECT
    FORMAT_DATE('%Y-%m', date(ts)) AS month,
    publisher,
    url,
    COUNT(1) AS pageviews

    FROM
    `ozone-analytics-dev.ozone.dim_pages`

    LEFT JOIN
    ozone.dim_publisher_mapping USING (publisher_id)

    WHERE
    date(ts) BETWEEN '2024-06-01' AND '2024-07-31' 
    AND RAND() < 0.1  -- 10% sampling condition

    GROUP BY
    1, 2, 3

    ORDER BY
    1
"""
# Run the query
query_job = client.query(query)

# Wait for the query to complete
result = query_job.result()  # Blocking wait for query to complete

# Print the job state
print(f"Query job state: {query_job.state}")

# Check for errors
if query_job.errors:
    print("Errors occurred during query execution:")
    for error in query_job.errors:
        print(error['message'])
else:
    print("No errors found. Query executed successfully.")

# Check the number of rows returned
total_rows = result.total_rows
print(f"Total rows returned: {total_rows}")


Query job state: DONE
No errors found. Query executed successfully.
Total rows returned: 47245357


In [3]:
print(type(result))

<class 'google.cloud.bigquery.table.RowIterator'>


In [4]:
# get results in a pandas dataframe 

df = result.to_dataframe()

df.head(2)

,month,publisher,url,pageviews
0,2024-06,daily mail general trust,https://www.dailymail.co.uk/news/article-54351...,2
1,2024-06,venatus,https://www.op.gg/summoners/kr/Fukase%20Satosh...,1


In [5]:
df.shape

(47245357, 4)

# Use python functions to find unique URLs and overlap

In [6]:
import datasketches
import time

# Calculate the number of unique URLs per month
unique_urls_by_month = df.groupby('month')['url'].nunique()
unique_urls_by_month.head(2)

month
2024-06    23344487
2024-07    23900538
Name: url, dtype: int64

In [7]:


# Create a dictionary to hold sets of URLs for each month
url_sets = {}

# Fill the dictionary with sets of URLs for each month
for month in df['month'].unique():
    url_set = set(df[df['month'] == month]['url'])
    url_sets[month] = url_set

# Function to calculate overlap between two sets
def calculate_overlap(set1, set2):
    return len(set1.intersection(set2))

# Example: Calculate overlap between two specific months
month1, month2 = '2024-06', '2024-07'  
if month1 in url_sets and month2 in url_sets:
    overlap = calculate_overlap(url_sets[month1], url_sets[month2])
    print(f"Number of unique URLs overlap between {month1} and {month2}: {overlap}")

# Calculating overlap for all pairs of months
overlap_results = pd.DataFrame(columns=['Month1', 'Month2', 'Overlap'])

months = list(url_sets.keys())
for i in range(len(months)):
    for j in range(i + 1, len(months)):
        overlap = calculate_overlap(url_sets[months[i]], url_sets[months[j]])
        new_row = pd.DataFrame({
            'Month1': [months[i]],
            'Month2': [months[j]],
            'Overlap': [overlap]
        })
        overlap_results = pd.concat([overlap_results, new_row], ignore_index=True)

print(overlap_results)


Number of unique URLs overlap between 2024-06 and 2024-07: 9481262
    Month1   Month2  Overlap
0  2024-06  2024-07  9481262


# use theta sketch to guess the unique count of URLs per month

In [8]:
import datasketches
import pandas as pd

# Create a dictionary to hold sketches for each month
theta_sketches = {}

# Initialize Theta Sketches for each month
for month in df['month'].unique():
    sketch = datasketches.update_theta_sketch()
    urls = df[df['month'] == month]['url']
    for url in urls:
        sketch.update(str(url))  # Ensure to use strings for URLs
    theta_sketches[month] = sketch

# Function to calculate overlap between two sketches using theta_intersection
def calculate_overlap(sketch1, sketch2):
    # Create a new intersection object
    intersection = datasketches.theta_intersection()
    
    # Update the intersection with both sketches
    intersection.update(sketch1)
    intersection.update(sketch2)
    
    # Get the estimate of the intersection
    overlap_estimate = intersection.get_result().get_estimate()
    return overlap_estimate

# Example: Calculate overlap between two specific months
month1, month2 = '2024-06', '2024-07'  # Example months, replace with actual values
if month1 in theta_sketches and month2 in theta_sketches:
    overlap = calculate_overlap(theta_sketches[month1], theta_sketches[month2])
    print(f"Approximate overlap of unique URLs between {month1} and {month2}: {overlap}")

# Calculating overlap for all pairs of months
overlap_results = pd.DataFrame(columns=['Month1', 'Month2', 'Overlap'])

months = list(theta_sketches.keys())
for i in range(len(months)):
    for j in range(i + 1, len(months)):
        overlap = calculate_overlap(theta_sketches[months[i]], theta_sketches[months[j]])
        new_row = pd.DataFrame({
            'Month1': [months[i]],
            'Month2': [months[j]],
            'Overlap': [overlap] 
        })
        overlap_results = pd.concat([overlap_results, new_row], ignore_index=True)

print(overlap_results)


Approximate overlap of unique URLs between 2024-06 and 2024-07: 9131507.383689346
    Month1   Month2       Overlap
0  2024-06  2024-07  9.131507e+06


/var/folders/rn/8rj4lvq928sgfy1ysnv98p380000gq/T/ipykernel_7829/1280065391.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  overlap_results = pd.concat([overlap_results, new_row], ignore_index=True)


In [9]:
# calculate the union 

# Create a dictionary to hold sketches for each month
theta_sketches = {}

# Initialize Theta Sketches for each month
for month in df['month'].unique():
    sketch = datasketches.update_theta_sketch()
    urls = df[df['month'] == month]['url']
    for url in urls:
        sketch.update(str(url))  # Convert URLs to string for consistency
    theta_sketches[month] = sketch

# Function to calculate total uniques using theta_union
def calculate_total_uniques(sketch1, sketch2):
    # Create a new union object
    union = datasketches.theta_union()
    
    # Update the union with both sketches
    union.update(sketch1)
    union.update(sketch2)
    
    # Get the estimate of the total unique elements in the union
    total_uniques_estimate = union.get_result().get_estimate()
    return total_uniques_estimate

# Example: Calculate total unique URLs across two specific months
month1, month2 = '2024-06', '2024-07'  # Example months, replace with actual values
if month1 in theta_sketches and month2 in theta_sketches:
    total_uniques = calculate_total_uniques(theta_sketches[month1], theta_sketches[month2])
    print(f"Approximate total unique URLs across {month1} and {month2}: {total_uniques}")

# Calculate total uniques for all pairs of months
total_uniques_results = pd.DataFrame(columns=['Month1', 'Month2', 'TotalUniques'])

months = list(theta_sketches.keys())
for i in range(len(months)):
    for j in range(i + 1, len(months)):
        total_uniques = calculate_total_uniques(theta_sketches[months[i]], theta_sketches[months[j]])
        new_row = pd.DataFrame({
            'Month1': [months[i]],
            'Month2': [months[j]],
            'TotalUniques': [total_uniques]
        })
        total_uniques_results = pd.concat([total_uniques_results, new_row], ignore_index=True)

print(total_uniques_results)


Approximate total unique URLs across 2024-06 and 2024-07: 38044301.89574235
    Month1   Month2  TotalUniques
0  2024-06  2024-07  3.804430e+07


/var/folders/rn/8rj4lvq928sgfy1ysnv98p380000gq/T/ipykernel_7829/4283220688.py:45: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  total_uniques_results = pd.concat([total_uniques_results, new_row], ignore_index=True)


# Test using Dask DF instead of pandas

In [3]:
import pandas as pd
import dask.dataframe as dd

# Initialize a list to hold pandas DataFrames
dataframes = []

# Chunk size for processing
chunk_size = 10000

# Process the RowIterator in chunks
rows = []
for row in result:
    rows.append(dict(row))
    if len(rows) >= chunk_size:
        df = pd.DataFrame(rows)
        dataframes.append(df)
        rows = []  # Reset the list to process the next chunk

# Process any remaining rows
if rows:
    df = pd.DataFrame(rows)
    dataframes.append(df)

# Create a Dask DataFrame from the list of Pandas DataFrames
dask_df = dd.from_pandas(pd.concat(dataframes, ignore_index=True), npartitions=len(dataframes))

# Inspect the first few rows of the Dask DataFrame
print(dask_df.head())


KeyboardInterrupt: 